<a href="https://colab.research.google.com/github/lxxhxng/aipython/blob/main/Dacon_jeju.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings(action='ignore')

# Fixed Random-Seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Load Data

In [ ]:
# 데이터 불러오기
# train_df = pd.read_csv('./train.csv')
# test_df = pd.read_csv('./test.csv')

from google.colab import drive
drive.mount("/content/gdrive/")

train_df = pd.read_csv('/content/gdrive/MyDrive/IBA/제주특산물 가격 예측/train.csv')
test_df = pd.read_csv('/content/gdrive/MyDrive/IBA/제주특산물 가격 예측/test.csv')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


# Data Pre-Processing

In [ ]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x : int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x : int(x[8:10]))

test_df['year'] = test_df['timestamp'].apply(lambda x : int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x : int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x : int(x[8:10]))

In [ ]:
# 월 별 평균 가격
monthly_avg_price = train_df.groupby('month')['price(원/kg)'].mean()
train_df['monthly_avg_price'] = train_df['month'].map(monthly_avg_price)

# 일 별 평균 가격
daily_avg_price = train_df.groupby('day')['price(원/kg)'].mean()
train_df['daily_avg_price'] = train_df['day'].map(daily_avg_price)

In [ ]:
#학습에 사용하지 않을 변수들을 제거합니다
train_x = train_df.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)'])
train_y = train_df['price(원/kg)']

test_x = test_df.drop(columns=['ID', 'timestamp'])

In [ ]:
train_x = pd.get_dummies(train_x, columns=['item', 'corporation', 'location'])
test_x = pd.get_dummies(test_x, columns=['item', 'corporation', 'location'])
# 원핫

In [ ]:
train_x

,year,month,day,monthly_avg_price,daily_avg_price,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,2019,1,1,991.624483,1022.383107,0,0,0,0,1,1,0,0,0,0,0,1,0
1,2019,1,2,991.624483,1038.668678,0,0,0,0,1,1,0,0,0,0,0,1,0
2,2019,1,3,991.624483,1059.287079,0,0,0,0,1,1,0,0,0,0,0,1,0
3,2019,1,4,991.624483,1204.002051,0,0,0,0,1,1,0,0,0,0,0,1,0
4,2019,1,5,991.624483,1115.878974,0,0,0,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,2023,2,27,1044.750318,1166.348205,0,0,0,1,0,0,0,0,0,0,1,1,0
59393,2023,2,28,1044.750318,1189.766667,0,0,0,1,0,0,0,0,0,0,1,1,0
59394,2023,3,1,1446.844135,1022.383107,0,0,0,1,0,0,0,0,0,0,1,1,0
59395,2023,3,2,1446.844135,1038.668678,0,0,0,1,0,0,0,0,0,0,1,1,0


In [ ]:
test_x

,year,month,day,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,2023,3,4,0,0,0,0,1,1,0,0,0,0,0,1,0
1,2023,3,5,0,0,0,0,1,1,0,0,0,0,0,1,0
2,2023,3,6,0,0,0,0,1,1,0,0,0,0,0,1,0
3,2023,3,7,0,0,0,0,1,1,0,0,0,0,0,1,0
4,2023,3,8,0,0,0,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2023,3,27,0,0,0,1,0,0,0,0,0,0,1,1,0
1088,2023,3,28,0,0,0,1,0,0,0,0,0,0,1,1,0
1089,2023,3,29,0,0,0,1,0,0,0,0,0,0,1,1,0
1090,2023,3,30,0,0,0,1,0,0,0,0,0,0,1,1,0


In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
model = XGBRegressor()
model.fit(train_x, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
preds = model.predict(test_x)

ValueError: ignored

In [ ]:
submission = pd.read_csv('/content/gdrive/MyDrive/IBA/제주특산물 가격 예측/sample_submission.csv')
submission

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [ ]:
submission['answer'] = preds
submission

,ID,answer
0,TG_A_J_20230304,3364.252673
1,TG_A_J_20230305,3254.014287
2,TG_A_J_20230306,3254.014287
3,TG_A_J_20230307,3254.014287
4,TG_A_J_20230308,3271.713930
...,...,...
1087,RD_F_J_20230327,471.515482
1088,RD_F_J_20230328,471.515482
1089,RD_F_J_20230329,471.515482
1090,RD_F_J_20230330,471.515482


In [ ]:
# 'ID' 칼럼에서 마지막 두 자리 숫자가 5, 12, 19, 26인 행 찾기
rows_to_update = submission[submission['ID'].str[-2:].astype(int).isin([5, 12, 19, 26])]

# 해당 행의 'answer' 칼럼에 0 대입
submission.loc[rows_to_update.index, 'answer'] = 0

# 결과 확인
print(submission.head())

                ID       answer
0  TG_A_J_20230304  3364.252673
1  TG_A_J_20230305     0.000000
2  TG_A_J_20230306  3254.014287
3  TG_A_J_20230307  3254.014287
4  TG_A_J_20230308  3271.713930


In [ ]:
submission.to_csv('./baseline_submission.csv', index=False)

In [ ]:
submission

,ID,answer
0,TG_A_J_20230304,3168.919189
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,2422.566895
3,TG_A_J_20230307,3131.846436
4,TG_A_J_20230308,3060.843750
...,...,...
1087,RD_F_J_20230327,90.683571
1088,RD_F_J_20230328,392.862274
1089,RD_F_J_20230329,305.213959
1090,RD_F_J_20230330,1256.847412
